In [1]:
## TOPIC Modelling

In [2]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import Dataset
df= pd.read_csv('C:/Users/thuyt/Documents/GitHub/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df = df.drop(['rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1)

In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [6]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [7]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [8]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [9]:
## BASE MODEL



# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11460.211584152643
Perplexity:  1872.0453564862053
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [11]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [12]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))



Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -3713.0659524242615
Model Perplexity:  701.8302700421284


In [13]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.03,0.03,0.89,0.03,0.03,2
Doc1,0.03,0.03,0.88,0.03,0.03,2
Doc2,0.05,0.05,0.82,0.05,0.05,2
Doc3,0.04,0.31,0.56,0.04,0.04,2
Doc4,0.04,0.04,0.86,0.04,0.04,2
Doc5,0.03,0.03,0.88,0.03,0.03,2
Doc6,0.03,0.08,0.83,0.03,0.03,2
Doc7,0.04,0.04,0.85,0.04,0.04,2
Doc8,0.04,0.07,0.83,0.04,0.04,2
Doc9,0.03,0.05,0.87,0.03,0.03,2


In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
2     -93.490028  -67.190376       1        1  85.852225
1     -78.863625   80.882698       2        1   3.836426
0      38.360535 -147.491760       3        1   3.493570
4      66.153419  133.825333       4        1   3.408890
3      99.883118  -11.538127       5        1   3.408890, topic_info=    Category       Freq          Term      Total  loglift  logprob
25   Default   5.000000    assessment   5.000000  30.0000  30.0000
222  Default   6.000000     political   6.000000  29.0000  29.0000
233  Default  16.000000       program  16.000000  28.0000  28.0000
321  Default   4.000000         woman   4.000000  27.0000  27.0000
14   Default   3.000000        afghan   3.000000  26.0000  26.0000
..       ...        ...           ...        ...      ...      ...
293   Topic5   0.160045      strategy   3.475465   0.3008  -5.7806
18    Topic5   0.160045  aim increase   2.975589   0.4560  -5.7806
289   Topic5   0.160045   stakeholder   2.948542   0.4652  -5.7806
201   Topic5   0.160044           non   2.480080   0.6382  -5.7806
83    Topic5   0.160044     democracy   3.821210   0.2059  -5.7806

[279 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
12        1  0.856299        advocacy
14        2  0.644109          afghan
16        1  0.939490             aim
18        1  0.672136    aim increase
19        1  0.872342  aim strengthen
...     ...       ...             ...
313       1  0.866534           train
316       1  0.958505    transparency
319       1  0.827620            unit
321       1  0.481759           woman
321       2  0.481759           woman

[104 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 5, 4])

In [15]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,0.200154,0.200158,0.200134,0.200140,0.200146,0.200148,0.200137,0.200164,0.200151,0.200142,...,0.200136,0.200148,0.200144,0.200138,0.200136,0.200152,0.200153,0.200249,0.200146,0.200164
Topic1,0.200126,0.200456,0.200114,0.200163,0.200119,0.200120,0.200460,0.200227,0.200684,0.200115,...,0.200177,0.200230,0.200121,0.200123,0.200180,0.200123,0.200794,2.350524,0.200395,0.200138
Topic2,2.674451,5.969368,8.518323,4.285131,3.901553,2.355828,6.756137,3.171455,5.195889,5.434051,...,8.084078,2.878096,10.540609,4.711752,5.746661,2.149015,4.504606,2.276859,9.193650,2.563287
Topic3,0.200158,0.200161,0.200138,0.200143,0.200150,0.200152,0.200140,0.200168,0.200155,0.200146,...,0.200139,0.200152,0.200148,0.200142,0.200139,0.200156,0.200158,0.200257,0.200150,0.200174
Topic4,0.200158,0.200161,0.200138,0.200143,0.200150,0.200152,0.200140,0.200168,0.200155,0.200146,...,0.200139,0.200152,0.200148,0.200142,0.200139,0.200156,0.200157,0.200257,0.200150,0.200169


In [16]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,region,task,anti,program,woman,afghan,assessment,tax,program support,association,report,long,civic,democratic governance,political
Topic 1,afghan,woman,assessment,political,stakeholder,civic,program support,democratic governance,democratic,democracy,conduct,association,information,participation,program
Topic 2,project,government,corruption,program,local,improve,support,strengthen,public,capacity,reform,increase,aim,governance,service
Topic 3,woman,afghan,report,assessment,program,program support,tax,long,association,civic,democratic governance,corruption,political,project design,structure
Topic 4,woman,afghan,commercial,assessment,program,program support,tax,report,long,association,civic,democratic governance,political,corruption,project design


In [17]:
### Document Clustering with topic model as features ###
tt_matrix = best_lda_model.components_

In [18]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=5, random_state=0)
km.fit_transform(lda_output)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([df, cluster_labels], axis=1)

,uuid,region,country,proj_nm,st_date,end_date,awd_amount,awd_value,implementer,sector,intervention,intervention_type,proj_desc,proj_res,corruption,stability,ClusterLabel
0,1,South and Central Asia (SCA),Afghanistan,Assistance for Afghanistan's Anti-Corruption A...,2010,2013,e) $25 M-$50 M,26642989,Management Systems International (MSI),"Independent Agencies, Civil Society","Corruption Prevention, Public Awareness","Awareness, Accountability",The five components of this program seek to ta...,The program developed a working relationship w...,High Corruption,Unstable,0
1,2,South and Central Asia (SCA),Afghanistan,Rule of Law Stabilization Program - Formal Com...,2010,2012,e) $25 M-$50 M,33700000,TetraTech ARD,Rule of Law,"Court Administration, Professional Legal Educa...","Transparency, Awareness",The overall goal of the program was to support...,The project set up legal education training fo...,Very High Corruption,Very Unstable,0
2,3,South and Central Asia (SCA),Afghanistan,Afghanistan Rule of Law Stabilization Project ...,2011,2012,c) $5 M-$10 M,10000000,"Checchi and Company Consulting, Inc.",Rule of Law,Access to Justice,"Awareness, Accountability",The project focused on strengthening and impro...,"The project reached 37,583 participants. In Ph...",Very High Corruption,Very Unstable,4
3,4,South and Central Asia (SCA),Afghanistan,Political Party Assessment,2012,2012,a) <$1 M,NaN,"Checchi and Company Consulting, Inc.",Elections and Political Processes,Assessment,Assessment,The Political Party Assessment team aimed to r...,The assessment team found that the development...,Very High Corruption,Very Unstable,3
4,30,Europe and Eurasia (E&E),Albania,Justice Sector Strengthening Project (JuST),2010,2015,c) $5 M-$10 M,9012613,Chemonics,"Rule of Law, Civil Society","Legal/Institutional Strengthening/Reform, Cour...","Transparency, Accountability, Awareness",The Justice Sector Strengthening Project (JuST...,"As of September 30, 2012, the project installe...",High Corruption,Unstable,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,311,Middle East and North Africa (MENA),Yemen,Responsive Governance Project (RGP),2010,2015,e) $25 M-$50 M,43000000,Counterpart International,"Civil Society, Legislature",Legislative/Institutional Strengthening/Reform...,"Enforcement, Citizen/Media/Business Engagement...",The Yemen Responsive Governance Project (RGP) ...,"To date, the RGP has assisted in crafting Yeme...",High Corruption,Very Unstable,0
311,312,Middle East and North Africa (MENA),Yemen,Enhancing Government Effectiveness Project (EGE),2007,2010,b) $1 M-$5 M,2939048,Management Systems International (MSI),Governance,"Financial Management Systems, Public Information","Transparency, Accountability",The Enhancing Government Effectiveness (EGE) i...,This project contributed to and improved capac...,High Corruption,Very Unstable,0
312,313,Middle East and North Africa (MENA),Yemen,Basic Health Services Project (BHS),2006,2010,d) $10 M-$25 M,21610314.78,Pathfinder International,"Healthcare, Governance","Service Delivery, Internal Controls, Procurement","Accountability, Awareness, Participation",The Basic Health Services (BHS) project was d...,The renovations and refurbishing of health fac...,High Corruption,Very Unstable,4
313,314,Sub-Saharan Africa (SSA),Zambia,Zambia Threshold Project (ZTP),2006,2008,d) $10 M-$25 M,22735000,Chemonics,"Rule of Law, Private Sector","Trade Reforms, Business Environment, Corruptio...","Transparency, Participation",The program was designed to reduce administrat...,The program promoted greater transparency and ...,High Corruption,Unstable,4
